In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import sqlalchemy as sq
import re
from tqdm import tqdm

In [2]:
data = pd.read_json("C:\chinadump.json", encoding="utf-8" )

In [3]:
data.head()

,chinadump
0,"{'doc_date': '2019-02-21', 'doc_country': '俄罗斯..."
1,"{'doc_date': '2019-02-21', 'doc_country': '萨摩亚..."
2,"{'doc_date': '2019-02-21', 'doc_country': '巴巴多..."
3,"{'doc_date': '2019-02-21', 'doc_country': '多米尼..."
4,"{'doc_date': '2019-02-21', 'doc_country': '吉尔吉..."


In [4]:
nd = []

for i in data.index:
    d = data.iloc[i,0]
    nd.append(list(d.values()))

In [5]:
data = pd.DataFrame(nd)

In [6]:
data.columns = ['doc_date','doc_country','doc_content']

In [7]:
data.head()

,doc_date,doc_country,doc_content
0,2019-02-21,俄罗斯,\n 为纪念中俄建交70周年，四川省歌舞剧院倾心打造的舞剧《长风啸》于2019年2月19日...
1,2019-02-21,萨摩亚,\n 2019年2月18日，驻萨摩亚大使王雪峰会见援萨摩亚太平洋运动会体育场馆项目中期验收...
2,2019-02-21,巴巴多斯,\n 2019年2月21日，国务委员兼外交部长王毅在北京会见来华参加中国和加勒比建交国外交...
3,2019-02-21,多米尼克,\n\n 2019年2月21日，中国与安提瓜和巴布达、巴哈马、巴巴多斯、多米尼克、格林纳达...
4,2019-02-21,吉尔吉斯斯坦,\n 2019年2月21日，国务委员兼外交部长王毅在北京与吉尔吉斯斯坦外长艾达尔别科夫举行...


In [8]:
data.shape

(132036, 3)

In [45]:
count = CountVectorizer(analyzer="char_wb", ngram_range=(2,5), min_df=2, binary=True)

In [20]:
countries = np.unique(data["doc_country"].copy())

In [10]:
docs = data['doc_content'].copy()

In [11]:
docs[0]

'\n\u3000\u3000为纪念中俄建交70周年，四川省歌舞剧院倾心打造的舞剧《长风啸》于2019年2月19日晚在克里姆林宫剧院隆重上演。驻俄罗斯大使李辉和夫人史晓玲女士观看演出。\n\u3000\u3000该剧运用浪漫神话形式，寓意人类大爱和维护和平主题，受到观众热烈欢迎。演出结束后，李大使夫妇上台慰问全体演职人员，感谢他们在元宵佳节为中俄民众奉献了独具特色的中国舞台艺术产品。\n\n\u3000\u3000李大使表示，此次演出是中国地方文化走出去的有益尝试，为今年中俄两国共同庆祝建交70周年营造了良好氛围，为弘扬中华文化、增进两国人民相互了解发挥了积极作用。希望四川省歌舞剧院深入挖掘当地特色文化资源，创作出更多优秀剧目，继续进入俄罗斯演艺市场，不断提升中国文化在国际舞台的形象和影响力。最后，李大使和史晓玲女士再次向演职人员表示祝贺，并致以节日的问候。\n\n\n\n\n\xa0\n\xa0\n'

In [40]:
docs.shape

(132036,)

In [46]:
countries[:10]

array(['七十七国集团', '上海合作组织', '不丹', '不结盟运动', '世界动物卫生组织', '世界气象组织', '世界经济论坛',
       '东亚峰会（EAS）', '东亚－拉美合作论坛', '东南亚国家联盟（ASEAN）'], dtype=object)

In [47]:
countries.shape

(246,)

In [48]:
cdict = []

In [50]:
for i in countries:
    print(data.loc[data['doc_country']==i,:].shape)
    if data.loc[data['doc_country']==i,:].shape[0] < 2:
        continue
    text = data.loc[data['doc_country']==i,'doc_content'].copy()
    vocab = set(count.fit(text).get_feature_names())
    vocab = vocab.difference(cdict[-1])
    cdict.append(vocab)

(1, 3)
(454, 3)
(26, 3)
(15, 3)
(1, 3)
(1, 3)
(88, 3)
(71, 3)
(167, 3)
(953, 3)
(681, 3)
(81, 3)
(61, 3)
(4, 3)
(2, 3)
(31, 3)
(10, 3)
(27, 3)
(34, 3)
(23, 3)
(812, 3)
(280, 3)
(834, 3)
(838, 3)
(1255, 3)
(655, 3)
(774, 3)
(442, 3)
(704, 3)
(150, 3)
(281, 3)
(131, 3)
(23, 3)
(57, 3)
(552, 3)
(1161, 3)
(389, 3)
(12, 3)
(466, 3)
(394, 3)
(2134, 3)
(533, 3)
(625, 3)
(127, 3)
(286, 3)
(633, 3)
(620, 3)
(9, 3)
(365, 3)
(147, 3)
(171, 3)
(543, 3)
(2, 3)
(375, 3)
(403, 3)
(737, 3)
(466, 3)
(37, 3)
(1, 3)
(3, 3)
(488, 3)
(7, 3)
(8, 3)
(1303, 3)
(582, 3)
(486, 3)
(581, 3)
(215, 3)
(1645, 3)
(719, 3)
(891, 3)
(530, 3)
(402, 3)
(376, 3)
(568, 3)
(231, 3)
(927, 3)
(714, 3)
(495, 3)
(424, 3)
(30, 3)
(272, 3)
(693, 3)
(7, 3)
(36, 3)
(7, 3)
(374, 3)
(1102, 3)
(1243, 3)
(858, 3)
(9, 3)
(811, 3)
(473, 3)
(355, 3)
(597, 3)
(601, 3)
(324, 3)
(1195, 3)
(662, 3)
(396, 3)
(304, 3)
(32, 3)
(136, 3)
(3, 3)
(725, 3)
(364, 3)
(576, 3)
(523, 3)
(632, 3)
(3, 3)
(7, 3)
(273, 3)
(1, 3)
(1167, 3)
(719, 3)
(1271, 3)


KeyboardInterrupt: 

In [ ]:
for i in cdict:
    print(len(i))

In [ ]:
newset = set()

for i in cdict:
    newset = newset.union(i)

In [ ]:
len(newset)

In [16]:
dictionaries = [set(["orange","cat"])]

In [17]:
dictionaries[-1]

{'cat', 'orange'}

In [18]:
for x in tqdm(range(10,11)):
    d = set()
    
    for i in tqdm(docs.head(10000)):
        
        matches = re.finditer(pattern="(?=(\w{{{}}}))".format(x),string=i)
        results = set([str(match.group(1)) for match in matches])
        
#         print(len(results.difference(d)))
        d = d.union(results)
#         print(len(d))
#         print()
    dictionaries.append(d)



  0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/10000 [00:00<?, ?it/s]


  1%|▏         | 132/10000 [00:00<00:07, 1309.93it/s]


  2%|▏         | 184/10000 [00:00<00:10, 896.60it/s] 


  2%|▏         | 228/10000 [00:00<00:16, 588.67it/s]


  3%|▎         | 268/10000 [00:00<00:20, 467.02it/s]


  3%|▎         | 306/10000 [00:00<00:24, 387.96it/s]


  3%|▎         | 341/10000 [00:00<00:27, 346.41it/s]


  4%|▎         | 374/10000 [00:00<00:30, 317.96it/s]


  4%|▍         | 405/10000 [00:01<00:38, 247.20it/s]


  4%|▍         | 432/10000 [00:01<00:45, 209.61it/s]


  5%|▍         | 456/10000 [00:01<00:50, 190.59it/s]


  5%|▍         | 477/10000 [00:01<00:53, 178.24it/s]


  5%|▍         | 497/10000 [00:01<00:56, 169.42it/s]


  5%|▌         | 516/10000 [00:01<00:59, 159.01it/s]


  5%|▌         | 533/10000 [00:01<01:01, 154.75it/s]


  6%|▌         | 550/10000 [00:02<01:02, 151.94it/s]


  6%|▌         | 566/10000 [00:02<01:03, 148.69it/s]


  6%|▌         | 582/10000 [00:0

 16%|█▌        | 1569/10000 [00:17<03:35, 39.07it/s]


 16%|█▌        | 1573/10000 [00:17<03:39, 38.39it/s]


 16%|█▌        | 1577/10000 [00:17<03:39, 38.41it/s]


 16%|█▌        | 1581/10000 [00:17<03:40, 38.23it/s]


 16%|█▌        | 1585/10000 [00:17<03:38, 38.44it/s]


 16%|█▌        | 1589/10000 [00:17<03:38, 38.48it/s]


 16%|█▌        | 1593/10000 [00:17<03:38, 38.39it/s]


 16%|█▌        | 1597/10000 [00:17<03:39, 38.33it/s]


 16%|█▌        | 1601/10000 [00:18<03:41, 37.86it/s]


 16%|█▌        | 1605/10000 [00:18<03:40, 38.07it/s]


 16%|█▌        | 1610/10000 [00:18<03:35, 38.87it/s]


 16%|█▌        | 1615/10000 [00:18<03:34, 39.03it/s]


 16%|█▌        | 1620/10000 [00:18<03:31, 39.64it/s]


 16%|█▌        | 1624/10000 [00:18<03:31, 39.69it/s]


 16%|█▋        | 1628/10000 [00:18<03:35, 38.79it/s]


 16%|█▋        | 1632/10000 [00:18<03:34, 38.95it/s]


 16%|█▋        | 1636/10000 [00:18<03:34, 39.06it/s]


 16%|█▋        | 1640/10000 [00:19<03:39, 38.03it/s]


 16%|█▋   

 22%|██▏       | 2173/10000 [00:34<03:39, 35.71it/s]


 22%|██▏       | 2177/10000 [00:34<03:37, 35.94it/s]


 22%|██▏       | 2181/10000 [00:34<03:36, 36.09it/s]


 22%|██▏       | 2186/10000 [00:34<03:32, 36.70it/s]


 22%|██▏       | 2190/10000 [00:35<03:37, 35.90it/s]


 22%|██▏       | 2194/10000 [00:35<03:32, 36.79it/s]


 22%|██▏       | 2198/10000 [00:35<03:40, 35.44it/s]


 22%|██▏       | 2202/10000 [00:35<03:37, 35.94it/s]


 22%|██▏       | 2206/10000 [00:35<03:34, 36.30it/s]


 22%|██▏       | 2210/10000 [00:35<03:35, 36.23it/s]


 22%|██▏       | 2214/10000 [00:35<03:37, 35.72it/s]


 22%|██▏       | 2218/10000 [00:35<03:40, 35.27it/s]


 22%|██▏       | 2222/10000 [00:35<03:41, 35.15it/s]


 22%|██▏       | 2226/10000 [00:36<03:50, 33.66it/s]


 22%|██▏       | 2230/10000 [00:36<03:48, 34.01it/s]


 22%|██▏       | 2234/10000 [00:36<03:44, 34.56it/s]


 22%|██▏       | 2238/10000 [00:36<03:41, 35.11it/s]


 22%|██▏       | 2242/10000 [00:36<03:47, 34.08it/s]


 22%|██▏  

 28%|██▊       | 2757/10000 [00:52<04:12, 28.65it/s]


 28%|██▊       | 2760/10000 [00:52<04:24, 27.40it/s]


 28%|██▊       | 2763/10000 [00:52<04:17, 28.07it/s]


 28%|██▊       | 2766/10000 [00:52<04:13, 28.56it/s]


 28%|██▊       | 2770/10000 [00:53<04:07, 29.21it/s]


 28%|██▊       | 2774/10000 [00:53<03:57, 30.43it/s]


 28%|██▊       | 2778/10000 [00:53<03:59, 30.21it/s]


 28%|██▊       | 2782/10000 [00:53<04:58, 24.18it/s]


 28%|██▊       | 2785/10000 [00:53<05:25, 22.17it/s]


 28%|██▊       | 2788/10000 [00:53<05:38, 21.31it/s]


 28%|██▊       | 2791/10000 [00:54<05:22, 22.36it/s]


 28%|██▊       | 2794/10000 [00:54<05:01, 23.88it/s]


 28%|██▊       | 2798/10000 [00:54<04:40, 25.72it/s]


 28%|██▊       | 2802/10000 [00:54<04:24, 27.18it/s]


 28%|██▊       | 2805/10000 [00:54<04:24, 27.23it/s]


 28%|██▊       | 2808/10000 [00:54<05:24, 22.14it/s]


 28%|██▊       | 2811/10000 [00:54<05:52, 20.38it/s]


 28%|██▊       | 2814/10000 [00:55<05:34, 21.50it/s]


 28%|██▊  

 31%|███▏      | 3137/10000 [01:12<06:50, 16.71it/s]


 31%|███▏      | 3140/10000 [01:12<06:28, 17.63it/s]


 31%|███▏      | 3143/10000 [01:12<06:18, 18.13it/s]


 31%|███▏      | 3145/10000 [01:12<06:25, 17.77it/s]


 31%|███▏      | 3147/10000 [01:12<06:37, 17.26it/s]


 31%|███▏      | 3149/10000 [01:13<06:52, 16.62it/s]


 32%|███▏      | 3151/10000 [01:13<07:03, 16.17it/s]


 32%|███▏      | 3153/10000 [01:13<07:06, 16.05it/s]


 32%|███▏      | 3155/10000 [01:13<07:03, 16.17it/s]


 32%|███▏      | 3157/10000 [01:13<06:59, 16.33it/s]


 32%|███▏      | 3159/10000 [01:13<06:57, 16.40it/s]


 32%|███▏      | 3161/10000 [01:13<07:00, 16.25it/s]


 32%|███▏      | 3163/10000 [01:13<07:10, 15.88it/s]


 32%|███▏      | 3165/10000 [01:14<07:07, 16.00it/s]


 32%|███▏      | 3167/10000 [01:14<07:06, 16.02it/s]


 32%|███▏      | 3169/10000 [01:14<07:09, 15.91it/s]


 32%|███▏      | 3171/10000 [01:14<07:03, 16.14it/s]


 32%|███▏      | 3173/10000 [01:14<07:00, 16.23it/s]


 32%|███▏ 

...

 53%|█████▎    | 5342/10000 [03:20<04:30, 17.22it/s]


 53%|█████▎    | 5344/10000 [03:20<04:31, 17.17it/s]


 53%|█████▎    | 5346/10000 [03:20<04:31, 17.15it/s]


 53%|█████▎    | 5348/10000 [03:20<04:41, 16.51it/s]


 54%|█████▎    | 5350/10000 [03:20<04:33, 17.01it/s]


 54%|█████▎    | 5352/10000 [03:20<04:35, 16.85it/s]


 54%|█████▎    | 5354/10000 [03:20<04:44, 16.33it/s]


 54%|█████▎    | 5356/10000 [03:20<04:42, 16.45it/s]


 54%|█████▎    | 5358/10000 [03:21<04:59, 15.49it/s]


 54%|█████▎    | 5360/10000 [03:21<04:56, 15.64it/s]


 54%|█████▎    | 5362/10000 [03:21<04:46, 16.19it/s]


 54%|█████▎    | 5364/10000 [03:21<04:47, 16.11it/s]


 54%|█████▎    | 5366/10000 [03:21<04:42, 16.39it/s]


 54%|█████▎    | 5368/10000 [03:21<04:34, 16.87it/s]


 54%|█████▎    | 5370/10000 [03:21<04:51, 15.89it/s]


 54%|█████▎    | 5372/10000 [03:21<04:50, 15.94it/s]


 54%|█████▎    | 5374/10000 [03:22<04:45, 16.22it/s]


 54%|█████▍    | 5376/10000 [03:22<04:49, 15.96it/s]


 54%|█████

 56%|█████▋    | 5640/10000 [03:38<05:01, 14.46it/s]


 56%|█████▋    | 5642/10000 [03:39<05:07, 14.17it/s]


 56%|█████▋    | 5644/10000 [03:39<05:09, 14.07it/s]


 56%|█████▋    | 5646/10000 [03:39<05:01, 14.45it/s]


 56%|█████▋    | 5648/10000 [03:39<04:47, 15.11it/s]


 56%|█████▋    | 5650/10000 [03:39<04:53, 14.80it/s]


 57%|█████▋    | 5652/10000 [03:39<05:01, 14.43it/s]


 57%|█████▋    | 5654/10000 [03:39<05:10, 14.01it/s]


 57%|█████▋    | 5656/10000 [03:40<05:15, 13.75it/s]


 57%|█████▋    | 5658/10000 [03:40<05:14, 13.80it/s]


 57%|█████▋    | 5660/10000 [03:40<05:14, 13.81it/s]


 57%|█████▋    | 5662/10000 [03:40<05:05, 14.18it/s]


 57%|█████▋    | 5664/10000 [03:40<05:12, 13.87it/s]


 57%|█████▋    | 5666/10000 [03:40<05:10, 13.97it/s]


 57%|█████▋    | 5668/10000 [03:40<05:15, 13.73it/s]


 57%|█████▋    | 5670/10000 [03:41<05:12, 13.87it/s]


 57%|█████▋    | 5672/10000 [03:41<05:05, 14.18it/s]


 57%|█████▋    | 5674/10000 [03:41<04:53, 14.74it/s]


 57%|█████

 59%|█████▉    | 5938/10000 [03:59<04:29, 15.07it/s]


 59%|█████▉    | 5940/10000 [03:59<04:31, 14.94it/s]


 59%|█████▉    | 5942/10000 [03:59<04:32, 14.88it/s]


 59%|█████▉    | 5944/10000 [03:59<04:33, 14.84it/s]


 59%|█████▉    | 5946/10000 [03:59<04:33, 14.84it/s]


 59%|█████▉    | 5948/10000 [03:59<04:32, 14.85it/s]


 60%|█████▉    | 5950/10000 [03:59<04:29, 15.05it/s]


 60%|█████▉    | 5952/10000 [03:59<04:27, 15.16it/s]


 60%|█████▉    | 5954/10000 [04:00<04:22, 15.42it/s]


 60%|█████▉    | 5956/10000 [04:00<04:19, 15.56it/s]


 60%|█████▉    | 5958/10000 [04:00<04:14, 15.86it/s]


 60%|█████▉    | 5960/10000 [04:00<04:19, 15.57it/s]


 60%|█████▉    | 5962/10000 [04:00<04:24, 15.24it/s]


 60%|█████▉    | 5964/10000 [04:00<04:25, 15.22it/s]


 60%|█████▉    | 5966/10000 [04:00<04:30, 14.92it/s]


 60%|█████▉    | 5968/10000 [04:01<04:33, 14.74it/s]


 60%|█████▉    | 5970/10000 [04:01<04:33, 14.74it/s]


 60%|█████▉    | 5972/10000 [04:01<04:36, 14.58it/s]


 60%|█████

 62%|██████▏   | 6236/10000 [04:18<04:00, 15.64it/s]


 62%|██████▏   | 6238/10000 [04:18<03:58, 15.78it/s]


 62%|██████▏   | 6240/10000 [04:18<03:58, 15.78it/s]


 62%|██████▏   | 6242/10000 [04:18<03:51, 16.25it/s]


 62%|██████▏   | 6244/10000 [04:19<04:00, 15.64it/s]


 62%|██████▏   | 6246/10000 [04:19<04:02, 15.45it/s]


 62%|██████▏   | 6248/10000 [04:19<04:03, 15.43it/s]


 62%|██████▎   | 6250/10000 [04:19<04:02, 15.43it/s]


 63%|██████▎   | 6252/10000 [04:19<03:59, 15.62it/s]


 63%|██████▎   | 6254/10000 [04:19<03:53, 16.02it/s]


 63%|██████▎   | 6256/10000 [04:19<04:03, 15.37it/s]


 63%|██████▎   | 6258/10000 [04:19<03:59, 15.63it/s]


 63%|██████▎   | 6260/10000 [04:20<04:02, 15.40it/s]


 63%|██████▎   | 6262/10000 [04:20<03:57, 15.74it/s]


 63%|██████▎   | 6264/10000 [04:20<04:00, 15.50it/s]


 63%|██████▎   | 6266/10000 [04:20<04:04, 15.28it/s]


 63%|██████▎   | 6268/10000 [04:20<04:02, 15.37it/s]


 63%|██████▎   | 6270/10000 [04:20<03:54, 15.93it/s]


 63%|█████

KeyboardInterrupt: 

In [ ]:
for i in dictionaries:
    print(len(i))

In [ ]:
for d in tqdm(dictionaries):
    dictionary = dictionary.union(d)

In [ ]:
dictionaries[:10]

In [ ]:
dictionary

In [ ]:
for i in tqdm(range(10)):
    print(i)